In [13]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
from datetime import datetime

# Fetch stock data using yfinance (example: AAPL, MSFT, GOOGL, AMZN, META)
start_date = '2023-02-24'
end_date = '2024-03-24'
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NFLX', 'NVDA', 'BABA', 'JPM']  # Adding more tickers

stock_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Calculate daily returns
stock_returns = stock_data.pct_change().dropna()

# Fetch Fama-French 5-factor data
# Download and clean Fama-French 5-factor data
ff_factors_url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip'
ff_factors = pd.read_csv(ff_factors_url, skiprows=3)

# Rename columns to match expectations
ff_factors.columns = ['Date', 'MKT', 'SMB', 'HML', 'RMW', 'CMA', 'RF']

# Convert Date column to datetime format and set as index
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)

# Align data by date and ensure proper date range
ff_factors = ff_factors.loc[start_date:end_date] / 100  # Convert percentages to decimals
data = pd.merge(stock_returns, ff_factors, left_index=True, right_index=True, how='inner')



[*********************100%%**********************]  10 of 10 completed


In [14]:
# Function to run Fama-French 5-factor regression for each stock
def run_ff_regression(data):
    results = {}
    tickers = stock_returns.columns
    
    for ticker in tickers:
        Y = data[ticker] - data['RF']  # Excess returns
        X = data[['MKT', 'SMB', 'HML', 'RMW', 'CMA']]
        X = sm.add_constant(X)
        
        model = sm.OLS(Y, X).fit()
        
        results[ticker] = {
            'alpha': model.params['const'],
            'beta_MKT': model.params['MKT'],
            'beta_SMB': model.params['SMB'],
            'beta_HML': model.params['HML'],
            'beta_RMW': model.params['RMW'],
            'beta_CMA': model.params['CMA'],
            'R-squared': model.rsquared_adj
        }
        
    return results

# Run the regression
ff_results = run_ff_regression(data)

# Display results for the first stock
first_ticker = list(stock_returns.columns)[0]
print(f"Results for {first_ticker}:")
print(ff_results[first_ticker])


Results for AAPL:
{'alpha': -0.0006941283419771295, 'beta_MKT': 1.1265681346107974, 'beta_SMB': 0.01961713377579729, 'beta_HML': -0.6129476852056667, 'beta_RMW': 0.4325087501921441, 'beta_CMA': 0.24029711600169545, 'R-squared': 0.4934691255006598}


In [15]:
# Validate model accuracy by calculating the mean squared error (MSE) of the residuals
def calculate_mse(data, results):
    mse_results = {}
    
    for ticker in stock_returns.columns:
        Y = data[ticker] - data['RF']
        X = data[['MKT', 'SMB', 'HML', 'RMW', 'CMA']]
        X = sm.add_constant(X)
        
        model = sm.OLS(Y, X).fit()
        residuals = model.resid
        mse = np.mean(residuals**2)
        
        mse_results[ticker] = mse
        
    return mse_results

mse_results = calculate_mse(data, ff_results)
print("MSE results:")
print(mse_results)


MSE results:
{'AAPL': 7.526059946002572e-05, 'AMZN': 0.00016244166598315638, 'BABA': 0.00050947776810915, 'GOOGL': 0.00018566015093842304, 'JPM': 8.295444104426813e-05, 'META': 0.00028848039806327156, 'MSFT': 7.879527086393478e-05, 'NFLX': 0.00038623396188650246, 'NVDA': 0.0004297378945942693, 'TSLA': 0.0006036672793698742}


In [17]:
from scipy.optimize import minimize

# Define a function to calculate portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Define constraints and bounds for the optimizer
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})  # Sum of weights must be 1
bounds = tuple((0, 1) for _ in range(len(stock_returns.columns)))  # Weights between 0 and 1

# Calculate the covariance matrix of the returns
cov_matrix = stock_returns.cov()

# Optimize the portfolio
def optimize_portfolio(cov_matrix):
    num_assets = len(stock_returns.columns)
    initial_guess = num_assets * [1. / num_assets,]
    
    result = minimize(portfolio_variance, initial_guess, args=(cov_matrix,),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result

opt_result = optimize_portfolio(cov_matrix)

# Optimized portfolio weights
opt_weights = opt_result.x
print("Optimized portfolio weights:")
print(opt_weights)


Optimized portfolio weights:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In [18]:
# Conduct scenario analysis by simulating changes in factor returns
def simulate_scenario(data, opt_weights, scenario_factors):
    # Apply scenario factors to the Fama-French factors
    scenario_data = data.copy()
    for factor, change in scenario_factors.items():
        scenario_data[factor] += change
        
    # Calculate expected portfolio return under the scenario
    portfolio_returns = scenario_data[stock_returns.columns].dot(opt_weights)
    expected_return = np.mean(portfolio_returns)
    return expected_return

# Define a scenario (e.g., a 1% increase in market return)
scenario_factors = {'MKT': 0.01, 'SMB': 0.005, 'HML': 0.002, 'RMW': -0.003, 'CMA': 0.004}

# Run scenario analysis
expected_return_scenario = simulate_scenario(data, opt_weights, scenario_factors)
print(f"Expected portfolio return under the scenario: {expected_return_scenario:.4f}")


Expected portfolio return under the scenario: 0.0021


In [19]:
# Generate a comprehensive report
def generate_report(ff_results, mse_results, opt_weights, expected_return_scenario):
    report = ""
    report += "Fama-French 5-Factor Model Results\n"
    report += "=" * 50 + "\n\n"
    
    for ticker, result in ff_results.items():
        report += f"Results for {ticker}:\n"
        report += "\n".join([f"{key}: {value:.4f}" for key, value in result.items()])
        report += "\n" + "-" * 50 + "\n"
    
    report += "\nModel Validation (MSE)\n"
    report += "=" * 50 + "\n"
    for ticker, mse in mse_results.items():
        report += f"{ticker}: {mse:.6f}\n"
    
    report += "\nOptimized Portfolio Weights\n"
    report += "=" * 50 + "\n"
    for ticker, weight in zip(stock_returns.columns, opt_weights):
        report += f"{ticker}: {weight:.4f}\n"
    
    report += "\nScenario Analysis\n"
    report += "=" * 50 + "\n"
    report += f"Expected portfolio return under the scenario: {expected_return_scenario:.4f}\n"
    
    return report

report = generate_report(ff_results, mse_results, opt_weights, expected_return_scenario)
print(report)

# Optionally, save the report to a file
with open('portfolio_analysis_report.txt', 'w') as f:
    f.write(report)


Fama-French 5-Factor Model Results

Results for AAPL:
alpha: -0.0007
beta_MKT: 1.1266
beta_SMB: 0.0196
beta_HML: -0.6129
beta_RMW: 0.4325
beta_CMA: 0.2403
R-squared: 0.4935
--------------------------------------------------
Results for AMZN:
alpha: 0.0004
beta_MKT: 1.0206
beta_SMB: -0.3162
beta_HML: -0.0298
beta_RMW: -0.2847
beta_CMA: -1.7414
R-squared: 0.5320
--------------------------------------------------
Results for BABA:
alpha: -0.0011
beta_MKT: 0.8879
beta_SMB: 0.0781
beta_HML: 0.0572
beta_RMW: -0.6312
beta_CMA: 0.2994
R-squared: 0.1131
--------------------------------------------------
Results for GOOGL:
alpha: 0.0002
beta_MKT: 1.1833
beta_SMB: -0.2739
beta_HML: -0.3642
beta_RMW: 0.2615
beta_CMA: -0.4947
R-squared: 0.3761
--------------------------------------------------
Results for JPM:
alpha: 0.0007
beta_MKT: 0.9288
beta_SMB: -0.3135
beta_HML: 0.7515
beta_RMW: -0.1683
beta_CMA: 0.3361
R-squared: 0.4545
--------------------------------------------------
Results for META:
alp